In [7]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import math
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from operator import itemgetter

from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

#Importing three component ensembles
from sklearn.ensemble import VotingClassifier

#importing SVC for second-step classification
from sklearn.svm import SVC

In [2]:
mutationRate = 0.001
crossOverRate = 0.06
iterations = 10
poolSize = 50

In [3]:
#Defining functions for genetic algorithms

def roulette(fitnessArray):
    index = 0
    cumalativeFitness = 0.0
    r = random.random()
    
    for i in range(len(fitnessArray)):
        cumalativeFitness += fitnessArray[i]
        if cumalativeFitness > r:
            return i


def selectFittest(fitness, rankedPool):
    while True:
        idx1 = roulette(fitness)
        idx2 = roulette(fitness)
        
        if idx1 is None or idx2 is None:
            continue
        elif idx1==idx2:
            continue
        else:
            break
    
    return rankedPool[idx1], rankedPool[idx2]

def crossover(chromosome1, chromosome2):
    randomSplitPoint = random.randint(1, len(chromosome1))
    return np.concatenate((chromosome1[:randomSplitPoint],chromosome2[randomSplitPoint:])), np.concatenate((chromosome2[:randomSplitPoint],chromosome1[randomSplitPoint:]))


def mutate(chromosome):

    mutatedChrom = []
    for ch in chromosome:
        if random.random()<mutationRate:
            if ch==1:
                mutatedChrom.append(0)
            else:
                mutatedChrom.append(1)
        else:
            mutatedChrom.append(ch)
    return mutatedChrom
    
def breed(chrome1, chrome2):
    if random.random()<crossOverRate:
        newChrome1, newChrome2 = crossover(chrome1, chrome2)
    else:
        newChrome1 = chrome1
        newChrome2 = chrome2
        
    newChrome1 = mutate(newChrome1)
    newChrome2 = mutate(newChrome2)
    
    return newChrome1, newChrome2

In [4]:
def rankPop(pool, X, y, classifier,fitnessFunction):
    scores = []
    for chromosome in pool:
        classifier = RandomForestClassifier()
        chosen_idx = [idx for gene, idx in zip(chromosome, range(X.shape[1])) if gene==1]
        if len(chosen_idx)==0:
            continue
        chosenX = X.iloc[:, chosen_idx]
        #performing leave-one out validation for instances less than 100
        #and 10 fold validation for others
        npoints = X.shape[0]
   
        if npoints <= 100:
            kf = KFold(n_splits = npoints)
        else:
            kf = KFold(n_splits = 10)
        
        kf.get_n_splits(X)
        classifier.fit(chosenX, y)
        train_X = []
        train_Y  = []
        prediction   = []
        predict_prob = []
        chosenX = np.array(chosenX)
        Y = np.array(y)
        for train_index, test_index in kf.split(X):
            classifier = RandomForestClassifier()
            for i in train_index:
                train_X.append(chosenX[i])
                train_Y.append(Y[i])

            classifier.fit(train_X, train_Y)
            for j in test_index:
                prediction.append(classifier.predict([chosenX[j]])[0])
                predict_prob.append(classifier.predict_proba([chosenX[j]])[0][1])
            train_X  = []
            train_Y  = []
        
   
        if(fitnessFunction == 'f-measure'):
            scores.append(f1_score(y_true=y,y_pred=prediction))
        elif(fitnessFunction == 'g-mean'):
            gScore = math.sqrt(precision_score(y_true = y, y_pred=prediction)*recall_score(y_true = y, y_pred =prediction ))
            scores.append(gScore)
        elif(fitnessFunction == 'accuracy'):
            scores.append(accuracy_score(y_true = y, y_pred = prediction))
        
    fitness = [x/sum(scores) for x in scores]
    pairedPop = zip(pool, fitness)
    rankedPop = sorted(pairedPop, key=itemgetter(-1), reverse = True)
    
    return rankedPop

In [5]:
def iteratePop(rankedPop):
    fitness = [item[-1] for item in rankedPop]
    rankedPool = [item[0] for item in rankedPop]
   
    new_pool = []
    new_pool.extend(rankedPool[:int(poolSize/15)])
    
    while(len(new_pool)<poolSize):
        ch1, ch2 = selectFittest(fitness, rankedPool)
        ch1, ch2 = breed(ch1, ch2)
        
        new_pool.append(ch1)
        new_pool.append(ch2)
    
    return new_pool[:poolSize]

In [6]:
def geneticAlgoFit(datafile,measure):
    datafile = pd.read_csv(datafile, dtype={'buggy':np.bool})
    X     = datafile.iloc[ : , :-1]
    y = datafile['buggy']

    pool = np.random.randint(0, 2, (poolSize, X.shape[1]))  
    for iteration in range(iterations):
#         print iteration
        classifier = RandomForestClassifier()
        rankedPop = rankPop(pool, X, y, classifier,measure)
#         print rankedPop
        pool = []
        pool = iteratePop(rankedPop)
        
    best_chromosome = rankPop(pool, X, y, classifier, measure)[0][0]
    return best_chromosome

In [8]:
directory = 'dataset/dataset/'
for fileName in os.listdir(directory):
    print(fileName)
    print(geneticAlgoFit(directory+fileName,'accuracy'))
    print(geneticAlgoFit(directory+fileName,'f-measure'))
    print(geneticAlgoFit(directory+fileName,'g-mean'))

camel-1.6.csv


KeyboardInterrupt: 

In [5]:
selectedFeaturesInEachSoftware = np.array([
    [[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1]],
    
    [[1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]],
    
    [[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]],
    
    [[0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]],
    
    [[1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]],
    
    [[0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]],
    
    [[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]],
    
    [[1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]],
    
    [[1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1],
    [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
    
    [[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0]],
    
    [[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]],
    
    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]],
    
    [[0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0]],
    
    [[1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1]],
    
    [[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]],
    
    [[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]],
    
    [[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
    [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
    [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1]],
    
    [[0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]],
    
    [[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
    [1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
    
    [[1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]],
    
    [[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]],
    
    [[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
    [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
    [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1]],
    
    [[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1]],
    
    [[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]]
])      

In [13]:
index = 0
directory = 'dataset/dataset/'
for fileName in os.listdir(directory):
    print fileName
    data = pd.read_csv(directory+fileName)
    X  = data.iloc[ : , :-1]
    Y = data.iloc[:,-1]
    accuracyFeature = selectedFeaturesInEachSoftware[index][0]
    fMeasureFeature = selectedFeaturesInEachSoftware[index][1]
    gMeanFeature = selectedFeaturesInEachSoftware[index][2]
   
    chosen_idx_accuracy = [idx for gene, idx in zip(accuracyFeature, range(X.shape[1])) if gene==1]
    chosen_idx_fMeasure = [idx for gene, idx in zip(fMeasureFeature, range(X.shape[1])) if gene==1]
    chosen_idx_gMean = [idx for gene, idx in zip(gMeanFeature, range(X.shape[1])) if gene==1]
    
    data['Accuracy_Pred'],data['Accuracy_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_accuracy],Y)
    data['FMeasure_Pred'],data['FMeasure_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_fMeasure],Y)
    data['GMean_Pred'],data['GMean_Pred_Prob'] = getPredictions(X.iloc[:,chosen_idx_gMean],Y)
    data.to_csv('dataset/annotated/'+fileName,index=False)    
    index+=1
    

camel-1.6.csv
e-learning.csv
intercafe.csv
ivy-2.0.csv
jedit-4.3.csv
kalkulator.csv
log4j-1.2.csv
lucene-2.4.csv
poi-2.5.csv
prop-6.csv
redaktor.csv
serapion.csv
sklebagd.csv
synapse-1.2.csv
systemdata.csv
szybkafucha.csv
termoproject.csv
tomcat.csv
velocity-1.6.csv
workflow.csv
wspomaganiepi.csv
xalan-2.7.csv
xerces-1.4.csv
zuzel.csv


In [11]:
def getPredictions(X,Y):
    
    base_learner1 = LogisticRegression(random_state=1)
    base_learner2 = DecisionTreeClassifier()
    base_learner3 = GaussianNB()

    classifier = VotingClassifier(estimators=[
                                 ('logregression', base_learner1), 
                                 ('dtree', base_learner2), 
                                 ('gnb', base_learner3)], 
                                  voting='soft')
    
    npoints = X.shape[0]
   
    if npoints <= 100:
        kf = KFold(n_splits = npoints)
    else:
        kf = KFold(n_splits = 10)
        
    kf.get_n_splits(X)
    train_X = []
    train_Y  = []
    prediction   = []
    predict_prob = []
    X = np.array(X)
    Y = np.array(Y)
    for train_index, test_index in kf.split(X):
        classifier = RandomForestClassifier()
        for i in train_index:
            train_X.append(X[i])
            train_Y.append(Y[i])

        classifier.fit(train_X, train_Y)
        for j in test_index:
            prediction.append(classifier.predict([X[j]])[0])
            predict_prob.append(classifier.predict_proba([X[j]])[0][1])
        train_X  = []
        train_Y  = []

    return prediction,predict_prob

In [22]:
def getSelectedFitnessFunction():
    i=1
    for fileName in os.listdir('dataset/annotated'):
        selectedFitnessFunction = [] 
        data = pd.read_csv('dataset/annotated/'+fileName)
        auc_acc = roc_auc_score(y_true=data['buggy'],y_score=data['Accuracy_Pred'])
        auc_fMeas = roc_auc_score(y_true=data['buggy'],y_score=data['FMeasure_Pred'])
        auc_gMean = roc_auc_score(y_true=data['buggy'],y_score=data['GMean_Pred'])
        
        print (auc_acc,auc_fMeas,auc_gMean)
        
        if auc_acc>auc_fMeas and auc_acc>auc_gMean:
            highest = 'Accuracy'
        elif auc_fMeas>auc_gMean and auc_fMeas>auc_acc:
            highest = 'FMeasure'
        else:
            highest = 'GMean'
        
        arr1 = np.array(data['Accuracy_Pred'])
        arr2 = np.array(data['FMeasure_Pred'])
        arr3 = np.array(data['GMean_Pred'])
        buggy = np.array(data['buggy'])
        
        for i in range(len(arr1)):
            flag = False
            if arr1[i]==buggy[i] and arr2[i] != buggy[i] and arr3[i] != buggy[i]:
                selectedFitnessFunction.append('Accuracy')
                flag = True
            elif arr1[i]!=buggy[i] and arr2[i] == buggy[i] and arr3[i] == buggy[i]:
                selectedFitnessFunction.append('FMeasure')
                flag = True
            elif arr1[i]!=buggy[i] and arr2[i] != buggy[i] and arr3[i] == buggy[i]:
                selectedFitnessFunction.append('GMean')
                flag = True
                
            if flag == False:
                selectedFitnessFunction.append(highest)
            
        data['Function to be selected'] = selectedFitnessFunction
#         print data.head()
        data.to_csv('dataset/annotated/'+fileName,index=False)


            

In [23]:
getSelectedFitnessFunction()

(0.5528457789096086, 0.562755004244366, 0.5547762808401105)
(0.4830508474576271, 0.4915254237288136, 0.6915254237288135)
(0.625, 0.625, 0.625)
(0.7214743589743591, 0.7229166666666667, 0.719551282051282)
(0.4979209979209979, 0.5433755433755433, 0.5454545454545454)
(0.8333333333333333, 0.6964285714285715, 0.8273809523809523)
(0.65625, 0.6197089947089947, 0.5831679894179894)
(0.7149868756966669, 0.6844234295782246, 0.724659307468268)
(0.8515569813986343, 0.8614463150459148, 0.860578055097716)
(0.6287878787878788, 0.6405723905723905, 0.6195286195286194)
(0.745836440467313, 0.730673626646781, 0.7206065125528213)
(0.6944444444444444, 0.6805555555555555, 0.5833333333333334)
(0.5625, 0.7916666666666666, 0.6666666666666667)
(0.8247606019151847, 0.8191176470588234, 0.7866963064295486)
(0.5932539682539683, 0.5376984126984127, 0.5376984126984127)
(0.6753246753246754, 0.5584415584415584, 0.6038961038961039)
(0.7771883289124668, 0.7984084880636604, 0.8328912466843501)
(0.5423449789646974, 0.55341137

In [34]:
def svctrain():
    directory = 'dataset/dataset/'
    annotated_directory = 'dataset/annotated/'
    DSF_directory = 'dataset/DSF/'
    for projectName in os.listdir(directory):
        print(projectName)
        projectData = pd.read_csv(directory + projectName)
        annotatedData = pd.read_csv(annotated_directory + projectName)
        
        #X contains software metrics and Y best ensemble selected
        X = np.array(projectData.iloc[ : , :-1])
        Y = np.array(annotatedData.iloc[ : , -1])
        
        npoints = X.shape[0]
        
        if npoints <= 100:
            kf = KFold(n_splits = npoints)
        else:
            kf = KFold(n_splits = 10)
        
        kf.get_n_splits(X)
        train_X = []
        train_Y = []
        
        predictedEnsemble = []
        predict_prob      = []
        final_prediction  = []  # this stores the prediction(bugginess) of the best ensemble predicted by SVC
        
        prediction_constant = '_Pred'
        probab_constant = '_Pred_Prob'
        
        
        for train_index, test_index in kf.split(X):
            classifier = SVC(probability = True)
            
            for i in train_index:
                train_X.append(X[i])
                train_Y.append(Y[i])
            
            unique_labels = np.unique(train_Y)
            if unique_labels.size == 1:
                for j in test_index:
                    predictedEnsemble.append(unique_labels[0])
                    predict_prob.append(annotatedData.loc[j, unique_labels[0] + probab_constant])
                    final_prediction.append(annotatedData.loc[j, unique_labels[0] + prediction_constant])
           
            else:
                classifier.fit(train_X, train_Y)
                
                for j in test_index:
                    predictedBestEnsemble = classifier.predict([X[j]])[0]
                    predictedEnsemble.append(predictedBestEnsemble)
                    final_prediction.append(annotatedData.loc[j, predictedBestEnsemble + prediction_constant])
                    
            # total probability of available classifiers, i.e the classifiers reported in unique_labels predicting true
                    predict_proba_true = 0
                    
            # probability of classifiers being predicted
                    predict_proba_classifiers = classifier.predict_proba([X[j]])[0]
                    k = 0
            # class probabilities are always reported in a sorted by name fashion, i.e AdaBoost, RandomForest, Voting 
            # np.unique also reports labels in a sorted by name fashion
                    for classifierName in unique_labels:
                        predict_proba_true +=  predict_proba_classifiers[k] * annotatedData.loc[j, classifierName + probab_constant]
                        k += 1
                    predict_prob.append(predict_proba_true)
                    
        annotatedData['PredictedTechnique'] = predictedEnsemble
        annotatedData['DSF_Prediction'] = final_prediction
        annotatedData['DSF_Pred_Prob'] = predict_prob
        annotatedData.to_csv(DSF_directory + projectName, index = False)

In [35]:
svctrain()

camel-1.6.csv
e-learning.csv
intercafe.csv
ivy-2.0.csv
jedit-4.3.csv
kalkulator.csv
log4j-1.2.csv
lucene-2.4.csv
poi-2.5.csv
prop-6.csv
redaktor.csv
serapion.csv
sklebagd.csv
synapse-1.2.csv
systemdata.csv
szybkafucha.csv
termoproject.csv
tomcat.csv
velocity-1.6.csv
workflow.csv
wspomaganiepi.csv
xalan-2.7.csv
xerces-1.4.csv
zuzel.csv


In [41]:
def computePerformanceMeasuresDSF():
    DSEdirectory = 'dataset/DSF/'
    projectMetrics = []
    index = 0
    projectMetrics = pd.DataFrame(projectMetrics,
                                    columns = ['Project','Precision', 'Recall', 'Auc_Score', 'Accuracy', 'Fmeasure', 'GMean'])
    for projectName in os.listdir(DSEdirectory):
        project = pd.read_csv(DSEdirectory + projectName)
        projectData = project.as_matrix(columns=[
                                         'DSF_Prediction',
                                         'DSF_Pred_Prob',
                                         'buggy'])
      
        row = []
        row.append(projectName)
        row.extend(computePerformanceMeasures(project['DSF_Prediction'], 
                                                         project['buggy'], 
                                                         project['DSF_Pred_Prob']))
        projectMetrics.loc[index] = row
        index = index + 1
  
    print(projectMetrics)
    projectMetrics.to_csv('dataset/' + 'results.csv', index = False)

In [42]:
computePerformanceMeasuresDSF()

              Project  Precision    Recall  Auc_Score  Accuracy  Fmeasure  \
0       camel-1.6.csv   0.678161  0.313830   0.732201  0.837306  0.429091   
1      e-learning.csv   0.666667  0.400000   0.738983  0.937500  0.500000   
2       intercafe.csv   1.000000  0.250000   0.793478  0.888889  0.400000   
3         ivy-2.0.csv   0.833333  0.625000   0.905489  0.897959  0.714286   
4       jedit-4.3.csv   1.000000  0.090909   0.710546  0.979675  0.166667   
5      kalkulator.csv   1.000000  0.916667   0.920635  0.969697  0.956522   
6       log4j-1.2.csv   0.945000  1.000000   0.674603  0.946341  0.971722   
7      lucene-2.4.csv   0.827957  0.758621   0.823739  0.761765  0.791774   
8         poi-2.5.csv   0.924107  0.834677   0.937765  0.888889  0.877119   
9          prop-6.csv   0.462963  0.378788   0.717401  0.893939  0.416667   
10       redaktor.csv   0.777778  0.518519   0.782128  0.903409  0.622222   
11       serapion.csv   1.000000  0.444444   0.833333  0.888889  0.615385   

In [40]:
def computePerformanceMeasures(predictions, labels, prediction_probability):
    
    precision = precision_score(y_true = labels, y_pred = predictions)
    recall    = recall_score(y_true = labels, y_pred = predictions)
    roc_score = roc_auc_score(labels, prediction_probability)
    accuracy  = accuracy_score(y_true = labels, y_pred = predictions)
    f_measure = 2*(precision * recall)/float(precision + recall) 
    g_mean = math.sqrt(precision * recall)
    
    metrics = [precision, recall, roc_score, accuracy, f_measure, g_mean]
    
    return metrics

In [60]:
def computePerformanceMeasuresForTechniques():
    annotated_directory = 'dataset/annotated/'
    resultsFile = open('dataset/performanceMeasures.csv','w+')
    for projectName in os.listdir(annotated_directory):
        project = pd.read_csv(annotated_directory+projectName)
        for techniques in ['Accuracy','FMeasure','GMean']:
            prediction = project[techniques+'_Pred']
            labels     = project['buggy']
            prediction_probability = project[techniques+'_Pred_Prob']
            measures = (','.join(str(x) for x in computePerformanceMeasures(prediction,labels,prediction_probability))+'\n')
            resultsFile.write(str(projectName)+','+techniques+','+measures)


In [61]:
computePerformanceMeasuresForTechniques()